# Unit Cube, Uniaxial Extension (Guccione)
This notebook will walk through a script that creates a unit cube governed passively by Guccione's strain energy function. The unit cube will be stretched along the fiber direction, and the stress response calculated.

In [1]:
import dolfin
import numpy as np # mainly for storing data
import fenics_plotly # for plotting finite element stuff
import matplotlib.pyplot as plt # for plotting traces

fenics_plotly.set_renderer("iframe") # renderer needed for jupyter notebooks

We are going to use numpy arrays to store any values we want to later plot (i.e. stress in the fiber direction). 
Big pictures steps in this script:  
- [Create mesh](#Create-Mesh)
- [Create necessary finite elements, function spaces, and functions](#Create-Finite-Elements,-Function-Spaces,-Functions)
- [Create boundary conditions](#Boundary-Conditions)
- Create weak form
- Solve weak form during some perturbation (in this case, updating right cube face displacement)
- "Hand Calculate" stress  

It can be seen how these steps are pretty general and can be their own modules, and will be what we follow in MyoFE.

### Create Mesh

In [2]:
mesh = dolfin.UnitCubeMesh(1,1,1)

### Create Finite Elements, Function Spaces, Functions

In [3]:
# Set up elements and spaces for the problem
# For displacement
Velem = dolfin.VectorElement("CG", mesh.ufl_cell(), 2, quad_scheme="default")
# mesh.ufl_cell() keeps this command generic, will always be 'tetrahedron' unless your mesh is 2D (which would give 'triangle')

# For hydrostatic pressure (enforcing incompressibility), of degree one less than element used for displacement
Qelem = dolfin.FiniteElement("CG", mesh.ufl_cell(), 1, quad_scheme="default")

# Mixed element function space, creating the mixed element inline using Velem and Qelem
W = dolfin.FunctionSpace(mesh, dolfin.MixedElement([Velem, Qelem]))

# For the weak form
w = dolfin.Function(W)
dw = dolfin.TrialFunction(W)
wtest = dolfin.TestFunction(W)

du, dp = dolfin.TrialFunctions(W)
u, p = dolfin.split(w)
v, q = dolfin.TestFunctions(W)

<mark> Kurtis, come down and explore what exactly dolfin.split(w) is doing, and why u is a 'ListTensor', and what exactly that means. Does manipulating values of u change w.sub(0), or is u just a copy of w.sub(0)? </mark>
In MyoFE, there will be more functions and function spaces for things at quadrature points, and myosim values like half-sarcomere length.

In [4]:
# Let's create a local coordinate system for the cube. 
# Right now, using Velem which has nodes defined for a CG2 element. In MyoFE, we define these at the quadrature points,
# so that when we solve MyoSim and assign active stress information in the f0 direction, there is no mismatch between
# myosim information at integration point i and the vector denoting f0.

# I think just assigning a continuous active stress field using the below V_space works, but there is instability when solving
# for active stress using something like MyoSim using this space. See work by Henrik Finsberg where they assign Ta using a
# space like this
V_space = dolfin.FunctionSpace(mesh,Velem)

f0 = dolfin.Function(V_space) 
s0 = dolfin.Function(V_space) 
n0 = dolfin.Function(V_space) 

for i in np.arange(np.shape(f0.vector())[0]/3.0):
    f0.vector()[3*i] = 1.0   # coincident with x-axis (gives vector (1,0,0) at each degree of freedom)
    s0.vector()[3*i+1] = 1.0 # coincident with y-axis (gives vector (0,1,0))
    n0.vector()[3*i+2] = 1.0 # coincident with z-axis (gives vector (0,0,1))

In MyoFE, the functions f0, s0, and n0 are created during mesh creation, and will be read in from the mesh file.  
Now we have a mesh, we have our finite elements defined on the mesh, and we have functions that will hold our solutions for displacement, hydrostatic pressure, and functions that represent our local coordinate system.  

### Boundary Conditions
This section will deal with assigning dirichlet boundary conditions. Neumann boundary conditions are in the weak form. There are multiple ways to do this. I'm going to show how we do it. We want the following dirichlet boundary conditions:
- fix the left face of the cube in the x-direction
- fix the lower face of the cube in the z-direction
- fix the back face of the cubeg in the y-direction

This should fix the point at the origin from moving at all, and allow for uniform expansion in the y- and z-directions.  
We will define classes that allow us to mark different faces of the cube as subdomains, that will be used in the dolfin DirichletBC() method.

In [8]:
class Left(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        return on_boundary and abs(x[0]) < tol

class Lower(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        return on_boundary and abs(x[2]) < tol
    
class Back(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        return on_boundary and abs(x[1]) < tol

class Fix(dolfin.SubDomain):
    def inside(self, x, on_boundary):
        tol = 1E-14
        return (dolfin.near(x[0],0.0,tol) and dolfin.near(x[1],0.0,tol) and dolfin.near(x[2],0.0,tol))

facetboundaries = dolfin.MeshFunction('size_t', mesh, mesh.topology().dim()-1)
facetboundaries.set_all(0)

N  = dolfin.FacetNormal(mesh)
ds = dolfin.ds(subdomain_data = facetboundaries)
dx = dolfin.dx(mesh,metadata = {"integration_order":2})

left  = Left()
lower = Lower()
back = Back()
#fix   = Fix()


left.mark(facetboundaries, 1)
lower.mark(facetboundaries, 2)
back.mark(facetboundaries, 3)
#fix.mark(facetboundaries, 3)

# Constrain left face in x
# W.sub(0) represents displacement (the first element in the above defined MixedElement), so W.sub(0).sub(0) is the 0th (or x) component of displacement
# We are using dolfin.Constant((0.0)) to denote that we want the chosen component of displacement to be zero
# and facetboundaries, 1 tells the method we want this to apply for the facetboundary labeled 1, which is the set of points belonging to the left class above
bc_left  = dolfin.DirichletBC(W.sub(0).sub(0), dolfin.Constant((0.0)), facetboundaries, 1)
# Constrain lower face in z
bc_lower = dolfin.DirichletBC(W.sub(0).sub(2), dolfin.Constant((0.0)), facetboundaries, 2)
# constrain back face in y
bc_back = dolfin.DirichletBC(W.sub(0).sub(1), dolfin.Constant((0.0)), facetboundaries, 3)
# Fixing point at origin to prevent rigid body translation
#bc_fix   = dolfin.DirichletBC(W.sub(0), dolfin.Constant((0.0, 0.0, 0.0)), fix, method="pointwise")

# Finally, our list of all dirichlet boundary conditions which will be given to our solver later
dirichlet_bcs = [bc_left, bc_lower, bc_back]